In [34]:
from konlpy.tag import Twitter
twitter = Twitter()

C:\Users\hans6\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [32]:
# 한국어 위키 데이터 load
import pandas as pd
doc = pd.read_excel("./data/글램+위피 후.xlsx")

In [33]:
data_date=doc['DATE'] # 리뷰 날짜
data_content=doc['REVIEW'] # 리뷰 콘텐츠

In [35]:
def review_preprocessing(data):
    # Hannanum package
    pos_tagger = Twitter()

    # 뉴스를 tokenizing한 후, 명사만 추출
    pos_nouns = pos_tagger.nouns(data)
    
    return ' '.join(pos_nouns)

In [36]:
content_noun=[]
for i in data_content:
    content_noun.append(review_preprocessing(i))

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

# ============================================
# -- Get TFIDF
# ============================================
vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(content_noun)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx
documents=[]
sentences=[]
values=[]
for i, sent in enumerate(content_noun):
    for token in sent.split():
        documents.append(i)
        sentences.append(token)
        values.append(sp_matrix[i, word2id[token]])
    
#     print('====== document[%d] ======' % i)
#     print( [ (token, sp_matrix[i, word2id[token]]) for token in sent.split() ] )

In [38]:
res_dict = []
for i in range(len(values)):
    res_dict.append({
        'documents' : documents[i],
        'sentences' : sentences[i],
        'values' : values[i]
    })
    
res_df = pd.DataFrame(res_dict)
res_df.head(20)

,documents,sentences,values
0,0,하트,0.424460
1,0,돈,0.000000
2,0,앱,0.000000
3,0,그,0.000000
4,0,이상,0.129301
5,0,이하,0.173479
6,0,를,0.000000
7,0,날,0.000000
8,0,상대방,0.243556
9,0,정보,0.142006


In [ ]:
res_df.to_excel('./data/글램+위피 후 TF-IDF.xlsx')

In [87]:
doc = pd.read_excel("./data/심쿵+당연시+아만다+정오 후 TF-IDF.xlsx")

In [88]:
doc2=pd.read_excel("./data/심쿵+당연시+아만다+정오 후.xlsx")

In [89]:
del doc['Unnamed: 0']
del doc2['Unnamed: 0']

In [90]:
doc.head(2)

,documents,sentences,values
0,0,다른앱보다,0.15672
1,0,데,0.00000


In [91]:
doc2.head(2)

,DATE,STAR,REVIEW
0,20201021,3,다른앱보다 괜찮은데 지금 문제가 심각한게 허위프로필로 사기치려는 계정이 심각하게 늘...
1,20201020,3,무료로 사용할 수 있다는 점이 굉장히 좋구요. 최근 업데이트를 해서인지 높은 평가해...


In [92]:
doc2['documents']=doc2.index

In [93]:
doc2.head(2)

,DATE,STAR,REVIEW,documents
0,20201021,3,다른앱보다 괜찮은데 지금 문제가 심각한게 허위프로필로 사기치려는 계정이 심각하게 늘...,0
1,20201020,3,무료로 사용할 수 있다는 점이 굉장히 좋구요. 최근 업데이트를 해서인지 높은 평가해...,1


In [94]:
doc3=pd.merge(doc,doc2, on='documents')

In [95]:
doc3.head(2)

,documents,sentences,values,DATE,STAR,REVIEW
0,0,다른앱보다,0.15672,20201021,3,다른앱보다 괜찮은데 지금 문제가 심각한게 허위프로필로 사기치려는 계정이 심각하게 늘...
1,0,데,0.00000,20201021,3,다른앱보다 괜찮은데 지금 문제가 심각한게 허위프로필로 사기치려는 계정이 심각하게 늘...


In [96]:
del doc3['DATE']
del doc3['REVIEW']

In [97]:
doc3['Weight STAR']=doc3['STAR']-2

In [98]:
doc3.head()

,documents,sentences,values,STAR,Weight STAR
0,0,다른앱보다,0.156720,3,1
1,0,데,0.000000,3,1
2,0,문제,0.137259,3,1
3,0,심각한,0.180053,3,1
4,0,허위프로필,0.180053,3,1


In [99]:
del doc3['STAR']

In [100]:
doc3['Weighted value']=doc3['values']*doc3['Weight STAR']

In [101]:
doc3[20:300]

,documents,sentences,values,Weight STAR,Weighted value
20,0,본인,0.137259,1,0.137259
21,0,어디업소,0.180053,1,0.180053
22,0,일,0.000000,1,0.000000
23,0,실장,0.180053,1,0.180053
24,0,티켓끊고나가란다,0.180053,1,0.180053
...,...,...,...,...,...
295,6,것,0.000000,0,0.000000
296,6,기존,0.271394,0,0.000000
297,6,얼평,0.139702,0,0.000000
298,6,꾸준히하,0.152141,0,0.000000


In [102]:
doc3.to_excel('심쿵+당연시+아만다+정오 후 긍정_부정.xlsx')